In [21]:
# Step 1: Import Necessary Libraries
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [22]:
# Initialize H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321.

.... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Temurin-17.0.12+7 (build 17.0.12+7, mixed mode, sharing)
  Starting server from C:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Huawei\AppData\Local\Temp\tmpcxu3dl5w
  JVM stdout: C:\Users\Huawei\AppData\Local\Temp\tmpcxu3dl5w\h2o_Huawei_started_from_python.out
  JVM stderr: C:\Users\Huawei\AppData\Local\Temp\tmpcxu3dl5w\h2o_Huawei_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Kuala_Lumpur
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 24 days
H2O_cluster_name:,H2O_from_python_Huawei_k1sdh4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.961 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [23]:
# Step 2: Load and Explore Data
# Load datasets
X_train = pd.read_csv("../04_modelling/dataset/X_train.csv")
y_train = pd.read_csv("../04_modelling/dataset/y_train.csv")
X_val = pd.read_csv("../04_modelling/dataset/X_val.csv")
y_val = pd.read_csv("../04_modelling/dataset/y_val.csv")
X_test = pd.read_csv("../04_modelling/dataset/X_test.csv")
y_test = pd.read_csv("../04_modelling/dataset/y_test.csv")

In [24]:
# Combine training features and target for H2O AutoML
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [25]:
# Convert to H2O Frames
train_h2o = h2o.H2OFrame(train_df)
val_h2o = h2o.H2OFrame(val_df)
test_h2o = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [26]:
# Define target and features
target = "yearly_compensation"
features = train_h2o.columns
features.remove(target)

In [27]:
# Check data summary
print("Training Data Summary:")
train_h2o.describe()

Training Data Summary:


Rows:5459
Cols:51

,ml_spending,used_tpu,Total_Experience,demographics_COUNTprofileTable,demographics_MEANprofileTable_yearly_compensation,company_COUNTprofileTable,company_MEANprofileTable_yearly_compensation,company_SUMprofileTable_yearly_compensation,country_COUNTprofileTable,country_SUMprofileTable_yearly_compensation,jobTitle_SUMprofileTable_yearly_compensation,jobTitle_SUMprofileTable_yearly_compensation_1,mlIncorporation_SUMprofileTable_yearly_compensation,mlIncorporation_MEANprofileTable_yearly_compensation,PrimaryTool_SUMprofileTable_yearly_compensation,PrimaryTool_MEANprofileTable_yearly_compensation,RecommendedLanguage_SUMprofileTable_yearly_compensation,RecommendedLanguage_MEANprofileTable_yearly_compensation,IncomeCategory_SUMprofileTable_yearly_compensation,age,gender_Male,company_size,country_Algeria,country_Australia,country_Austria,country_Nigeria,country_Other,country_Philippines,country_Poland,country_Portugal,country_Republic_of_Korea,country_Russia,country_Saudi_Arabia,country_Singapore,country_Spain,country_Switzerland,country_Taiwan,country_Thailand,country_Tunisia,country_Turkey,country_Ukraine,country_United_States_of_America,country_Viet_Nam,job_title_Business_Analyst,job_title_Data_Analyst,job_title_Data_Engineer,job_title_Data_Scientist,job_title_Product/Project_Manager,job_title_Research_Scientist,job_title_Software_Engineer,yearly_compensation
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,int
mins,-5.073214151002757,-4.986915629786675,-3.894034176423702,-4.626803917463832,-4.485160789575362,-4.697719812525946,-4.177191085597267,-4.489270330011596,-5.474562681728408,-4.141542218290927,-4.312797198084712,-3.640135585386583,-3.9156531768923344,-3.8090352815546336,-3.948373607951701,-6.633594637063434,-3.554863581010869,-3.6676969322076007,-4.58154777588761,-4.182098831765816,-4.945087032146247,-4.158391286783692,-4.129525464152223,-2.956127991412488,-3.309099336730281,-3.624885508492478,-4.969630300896252,-5.033903596696299,-3.5386477083792016,-4.420184254999469,-4.168450093896267,-4.125855910126183,-3.856805748582876,-4.107408970710599,-4.546294437727609,-3.791664434666988,-4.743266082092007,-3.6969835452361934,-4.476333448057696,-3.672761484346168,-3.3169862291510386,-2.990471642762147,-3.5763992584874185,-3.471249864028924,-2.514554877453015,-2.1795053058669813,-1.4953476643568728,-0.211262928713999,-0.1929108164798906,-0.1518463636541843,0.0
mean,-0.003377296744197873,-0.008395492689889886,-0.00012469694809002992,-0.01035154226995089,0.0037394333234852532,-0.0032719797218479976,-0.004223564022879654,-0.0030969986054931436,-0.007930048444353504,-0.014272447446954183,-0.011401307680057416,0.003381163321315585,-0.010496233527614333,-0.01008031830684686,-0.009583903537950623,0.0012660169500348264,0.009880261098446433,-0.00269898603019323,2.5584556922530697e-05,-0.0015476633985693098,-0.007047951781546629,0.0012954530505763725,0.012184540794283355,-0.005218221477880209,-0.003962401376109956,-0.0020463546342546057,-0.00423146190723855,-0.005157047548750889,0.007398474252734203,-0.006323640419355434,0.009063268330118057,-0.0022356159238555483,-0.0064200427519799375,-0.005383689458664042,-0.012404747101242718,0.01433426178874028,-0.009987254513806838,0.0008994458528694564,0.009436431220184109,-0.00414452204445641,-0.0030703035923405862,0.013519736896756748,-0.001578680102810357,-0.008244308868963176,0.006181237787700669,-0.004139823792625962,-0.0016791350125122035,-0.00036803019575168264,-0.0006041522292237774,-5.193256324336065e-05,10.382670818831283
maxs,7.269771021095883,7.460528071179854,6.782091584796228,7.441591933236912,6.002826818335985,4.806017107174871,4.496645242889465,5.000959400082108,6.018563119248007,4.642460463878098,4.799734330177638,4.576483809859401,4.152266969366401,4.177939068866628,5.422563471680

In [28]:
# Step 3: Model Selection using AutoML
# Configure AutoML parameters
h2o_automl = H2OAutoML(
    max_models=10,             # Limit the number of models to explore
    seed=10,                   # Set seed for reproducibility
    exclude_algos=["DeepLearning"],  # Exclude deep learning models
    verbosity="info",         # Enable detailed logs
    nfolds=0                  # Use validation set directly instead of k-fold CV
)

In [29]:
# Train the AutoML models with cross validation
h2o_automl.train(x=features, y=target, training_frame=train_h2o, validation_frame=val_h2o)

AutoML progress: |
20:28:44.931: Project: AutoML_1_20241226_202844
20:28:44.932: Cross-validation disabled by user: no fold column nor nfolds > 1.
20:28:44.932: Setting stopping tolerance adaptively based on the training frame: 0.0135345385323802
20:28:44.932: Build control seed: 10
20:28:44.933: training frame: Frame key: AutoML_1_20241226_202844_training_Key_Frame__upload_865b450262053ea35082fd086f784c9a.hex    cols: 51    rows: 5459  chunks: 2    size: 2195995  checksum: -217094960968714656
20:28:44.933: validation frame: Frame key: Key_Frame__upload_a00b139507a78235cb7c3896cefa4cda.hex    cols: 51    rows: 1170  chunks: 1    size: 472638  checksum: 742896563949582232
20:28:44.933: leaderboard frame: Frame key: Key_Frame__upload_a00b139507a78235cb7c3896cefa4cda.hex    cols: 51    rows: 1170  chunks: 1    size: 472638  checksum: 742896563949582232
20:28:44.934: blending frame: NULL
20:28:44.934: response column: yearly_compensation
20:28:44.934: fold column: null
20:28:44.934: weight

H2OJobCancelled: Job<$03017f00000132d4ffffffff$_a9cde2e84f9c647f3884662018b943fb> was cancelled by the user.

In [ ]:
# Review Leaderboard
h2o_models = h2o.automl.get_leaderboard(h2o_automl, extra_columns="All")
h2o_models

In [11]:
# Identify the best-performing model
best_model = h2o_automl.leader
print(f"Best Model: {best_model.model_id}")

Best Model: GLM_1_AutoML_1_20241226_201057


In [12]:
# Step 5: Evaluate on Test Data
# Generate predictions
predictions = best_model.predict(test_h2o)
actual_predictions = y_test

h2o_mean_squared_error(predictions, actual_predictions)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Display a preview of predictions
h2o_compare = pd.DataFrame(data={'actual': y_test, 'predicted': predictions.as_data_frame().to_numpy().ravel()})
h2o_compare.plot(figsize=(10, 6), title="Actual vs Predicted Yearly Compensation (H2O AutoML)", xlabel="Actual Vs Predicted", ylabel="Yearly Compensation")

Predictions on Test Data:


H2OConnectionError: Connection was closed, and can no longer be used.

In [14]:
# Evaluate model performance
performance = best_model.model_performance(test_h2o)
print("Model Performance on Test Data:")
print(performance)

Model Performance on Test Data:
ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.580239033646979
RMSE: 0.7617342276982039
MAE: 0.5441590865678606
RMSLE: NaN
Mean Residual Deviance: 0.580239033646979
R^2: 0.9898857737387018
Null degrees of freedom: 1169
Residual degrees of freedom: 1119
Null deviance: 67121.28489254977
Residual deviance: 678.8796693669655
AIC: 2787.467461661745


In [15]:
# Step 6: Advanced Techniques - Ensemble Model Analysis
# Extract specific models from the leaderboard for further analysis (e.g., Gradient Boosting)
model_ids = leaderboard['model_id'].as_data_frame()['model_id'].tolist()
gbm_model_id = next(mid for mid in model_ids if "GBM" in mid)
gbm_model = h2o.get_model(gbm_model_id)

c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [16]:
# Analyze GBM Model
gbm_performance = gbm_model.model_performance(test_h2o)
print("GBM Model Performance:")
print(gbm_performance)

GBM Model Performance:
ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 3.1661946202578672
RMSE: 1.7793804034713507
MAE: 1.307297046193763
RMSLE: NaN
Mean Residual Deviance: 3.1661946202578672


In [17]:
# Step 7: Save and Interpret Results
# Save leaderboard and best model locally for further analysis
# leaderboard_path = "../04_modelling/results/leaderboard.csv"
# leaderboard.as_data_frame().to_csv(leaderboard_path, index=False)
# print(f"Leaderboard saved to: {leaderboard_path}")

c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


OSError: Cannot save file into a non-existent directory: '..\04_modelling\results'

In [18]:
# Save the best model
best_model_path = h2o.save_model(best_model, path="../04_modelling/models/")
print(f"Best model saved to: {best_model_path}")

Best model saved to: C:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\notebooks\04_modelling\models\GLM_1_AutoML_1_20241226_201057


In [30]:
# Shutdown H2O cluster after processing
h2o.shutdown(prompt=False)

H2O session _sid_aefc closed.


C:\Users\Huawei\AppData\Local\Temp\ipykernel_22940\3898568403.py:2: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)
